In [1]:
import os
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from datasets import load_dataset
from sklearn import preprocessing

In [2]:
os.environ['WANDB_DISABLED'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
# train_df = pd.read_csv('data/intent_train_data.csv')
raw_dataset = load_dataset('csv', data_files='data/intent_train_data.csv')

Using custom data configuration default-66de2ebea2db25aa
Reusing dataset csv (C:\Users\ninja\.cache\huggingface\datasets\csv\default-66de2ebea2db25aa\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [4]:
le = preprocessing.LabelEncoder()
le.fit(raw_dataset['train']['intent'])

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

def text_tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
def intent_format_function(examples):
    return {'label': le.transform(examples['intent'])}

tokenized_dataset = raw_dataset.map(text_tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.map(intent_format_function, batched=True)

train_dataset = tokenized_dataset['train']

Loading cached processed dataset at C:\Users\ninja\.cache\huggingface\datasets\csv\default-66de2ebea2db25aa\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-6cc9172bf89a54c5.arrow
Loading cached processed dataset at C:\Users\ninja\.cache\huggingface\datasets\csv\default-66de2ebea2db25aa\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-8e78451ace62727c.arrow


In [5]:
train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'intent', 'label', 'text'],
    num_rows: 28
})

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased',
    num_labels = 2,
    cache_dir = './models')

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [10]:
training_args = TrainingArguments(
    output_dir = './models/intent_modeling',
    gradient_accumulation_steps = 1,
    learning_rate = 2e-5,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    label_names = ['label'],
    report_to = None,
    no_cuda = False,
    logging_steps = 5,
    save_steps = 20)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset)
    #batch_size = 1)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
# # trainer.compute_loss(model, train_dataset[0:2])
# dl = trainer.get_train_dataloader()
# train_dataset
# next(iter(dl))

# # import inspect
# # print(inspect.getsource(trainer.train))

In [ ]:
model

In [11]:
trainer.train()

Step,Training Loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ninja\.conda\envs\ml-env\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-3435b262f1ae>", line 1, in <module>
    trainer.train()
  File "C:\Users\ninja\.conda\envs\ml-env\lib\site-packages\transformers\trainer.py", line 1148, in train
    torch.nn.utils.clip_grad_norm_(
  File "C:\Users\ninja\.conda\envs\ml-env\lib\site-packages\torch\nn\utils\clip_grad.py", line 40, in clip_grad_norm_
    p.grad.detach().mul_(clip_coef.to(p.grad.device))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ninja\.conda\envs\ml-env\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exc

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()